# NLP Lab : Modèles de langue

Dans ce tp, nous allons constuire les briques principales du modèle GPT2 et entrainer un petit modèle sur des poèmes de Victor Hugo. 

Les questions sont posées dans ce notebook, puis pour executer l'entrainement, il faudra modifier le ficher `gpt_single_head.py` aussi disponible dans le reposository git.


## Données

Les données d'entrainement sont un recueil de poèmes de Victor Hugo issu du site [gutenberg.org](https://www.gutenberg.org/). Elles sont disponibles dans le répertoire `data`.

Afin de réduire la complexité du modèle, nous allons modéliser le texte au niveau caractère. Les modèles de language modélisent généralement des séquences de sous-mots en utilisant des [tokenizers](https://huggingface.co/docs/transformers/tokenizer_summary) (BPE, SentencePiece, WordPiece)

Questions :
>* En utilisant [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter), afficher le nombre de caractères différents dans le texte et la fréquence de chaque caractère.

In [1]:
import collections

with open('data/hugo_contemplations.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'Number of characters in the file: {len(text)}')
##  YOUR CODE HERE
counter = collections.Counter(text)
chars = set(text)

###

print (f'Number of character in counter: {sum(counter.values())}')
print (f'{len(chars)} different characters')
print (counter)


Number of characters in the file: 285222
Number of character in counter: 285222
101 different characters
Counter({' ': 49127, 'e': 30253, 's': 17987, 'u': 14254, 'r': 14223, 't': 14071, 'a': 14048, 'n': 13725, 'i': 12828, 'o': 12653, 'l': 11638, '\n': 8102, 'm': 6495, 'd': 6375, ',': 6077, 'c': 5074, 'p': 4206, "'": 3820, 'v': 3492, 'é': 2943, 'b': 2783, 'f': 2772, 'h': 2221, 'q': 1956, 'g': 1790, '.': 1420, 'x': 1154, 'L': 1147, '!': 1121, 'E': 1074, ';': 1043, '-': 1020, 'j': 890, 'D': 764, 'è': 725, 'à': 706, 'y': 660, 'I': 627, 'ê': 605, 'C': 593, 'S': 545, 'A': 530, 'Q': 503, 'z': 482, 'J': 471, 'O': 450, 'T': 441, 'P': 435, '?': 388, 'V': 383, 'â': 381, 'N': 362, 'M': 344, 'ù': 298, ':': 294, 'R': 240, 'î': 214, 'U': 208, 'ô': 159, 'X': 150, '1': 146, 'H': 116, 'F': 114, '5': 111, '8': 93, 'B': 78, '«': 74, 'É': 70, '»': 69, 'G': 67, '4': 64, 'û': 62, '3': 47, 'ç': 34, 'À': 33, 'ë': 32, 'ï': 31, '2': 30, '·': 26, 'Ê': 24, '6': 23, '7': 23, 'Ô': 19, '9': 19, 'È': 11, 'k': 10, '0':

### Encodage / décodage
Afin de transformer le texte en vecteur pour le réseau de neurones, il faut encoder chaque caractère avec un entier. Les fonctions suivantes opérent l'encodage et le décodage des caractères :

In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: transform a string into a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: transform a list of integers into a string


# test that your encoder/decoder is coherent
testString = "\nDemain, dès l'aube"
decode(encode (testString)) ==  testString

True

### Découpage Train/Validation

L'objectif étant de prédire des poèmes, il ne faut pas mélanger les lignes aléatoirement. Il faut garder l'ordre des lignes dans le texte et uniquement prendre les premiers 90% pour entrainer et les 10% restants pour contrôler l'apprentissage. 

Questions :
> * Découper en `train_data` (90%) et `val_data` (10%) en utilisant du slicing sur data.

In [3]:
import torch
# Train and validation splits
data = torch.tensor(encode(text), dtype=torch.long)
## YOUR CODE HERE
# first 90% will be train, rest val
split_index = int(len(data) * 0.9)
train_data = data[:split_index]
val_data = data[split_index:]
###

C:\Users\Dell\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Contexte

Le modèle de langue possède comme paramètre la taille maximale du contexte à considérer pour faire la prédiction du prochain caractère. Ce contexte est appelé `block_size`. Les données d'apprentissage sont donc des séquences de caractères consécutifs, issues de l'ensemble d'entraînenement tirées aléatoirement et de longueur `block_size`.

Si le caractère de début de la séquence est `i`, la séquence de contexte est donc :
``` x = data[i:i+block_size]```
et la valeur à prédire à chaque position dans le contexte est le caractère suivant :
```y = [data[i+1:i+block_size+1]```.



In [4]:
block_size = 8

i  = torch.randint(len(data) - block_size, (1,))
print (i)
x = train_data[i:i+block_size]
y = train_data[i+1:i+1+block_size]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print (f'context is >{decode(context.tolist())}< target is >{decode([target.tolist()])}<')

tensor([158327])
context is >l< target is >e<
context is >le< target is >s<
context is >les< target is > <
context is >les < target is >p<
context is >les p< target is >o<
context is >les po< target is >r<
context is >les por< target is >t<
context is >les port< target is >e<


### Définition des batchs

Les batchs d'entrainement sont constitués de plusieurs séquences de caractères tirées aléatoirement dans `train_data`. Pour choisir aléatoirement une séquence à mettre dans le batch, il faut tirer aléatoirement un point de départ dans `train_data` et extraire les `block_size` caractères suivants. Lors du tirage du point de départ, faire attention à laisser suffisamment de caractères après le point de départ pour avoir une séquence de `block_size` caractères.

Questions :
> * Créer les batchs `x` en tirant `batch_size` séquences de longeur `block_size` à  partir d'un point `i` tiré aléatoirement. Empiler les exemples avec `torch.stack`.
> * Créer les batchs `y` en ajoutant le caractère suivant la séquence `x`. Empiler les exemples avec `torch.stack`.




In [45]:
import random
batch_size = 4
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # select batch_size starting points in the data, store them in a list called starting_points
    starting_points = torch.randint(0, len(data)-block_size, (batch_size,))
    # x is the sequence of integer starting at each straing point and of length block_size
    x = [data[i:i+block_size] for i in starting_points]
    # y is the character after each starting position
    y = [data[i+1:i+block_size+1] for i in starting_points]
    x = torch.stack(x)
    y = torch.stack(y)
    
    # send data and target to device
    x, y = x.to(device), y.to(device)
    
    return x, y

### Premier modèle : un bigramme 

Le premier modèle que nous allons implémenter est un modèle bigramme. Il prédit le caractère suivant uniquement en fonction du caractère courant. Il est possible de stocker ce modèle dans une simple matrice : pour chaque caractère (en ligne), on stocke la distribution de probabilités sur l'ensemble des caractères suivants (en colonne). On peut donc le stocker dans une simple couche [`Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html).

Questions :
> * Dans le constructeur, définir une couche Embedding de taille `vocab_size` par `vocab_size`.
> * Dans forward, appliquer la couche d'embedding au batch de idx (`x`).
> * Dans forward, définir la loss comme la `cross_entropy` entre la prédiction et target (`y`).



In [46]:
import torch.nn as nn
vocab_size=101
# use a gpu if we have one
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # we use a simple vocab_size times vocab_size tensor to store the probabilities 
        # of each token given a single token as context in nn.Embedding
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, vocab_size)
        ## 
        
    def forward(self, idx, targets=None):

        # idx and targets are both (Batch,Time) tensor of integers
        # YOUR CODE HERE
    
        logits = self.embedding(idx)
   
        # don't compute loss if we don't have targets
        if targets is None:
            loss = None
        else:
            # change the shape of the logits and target to match what is needed for CrossEntropyLoss
            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            
            Batch, Time, Channels = logits.shape
            logits = logits.view(Batch*Time, Channels)
            targets = targets.view(Batch * Time)
            
            # negative log likelihood between prediction and target
            # YOUR CODE HERE
            loss = nn.CrossEntropyLoss()(logits, targets)
            ## 

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
# send the model to device
m = model.to(device)

#### Modèle avant entraînement
Le modèle n'a pas encore été entrainé, il est juste initialisé, mais on peut calculer la loss sur un batch aléatoire. Les poids étant initialisés avec une distribution normale N(0,1) sur chaque dimension, la loss attendue après l'initialisation devrait être proche de `-ln(1/vocab_size)` (l'entropie est maximale).

In [47]:
import math
xb, yb = get_batch('train')
logits, loss = m(xb, yb)
print (logits.shape)
print (f'loss attendue {-math.log(1.0/vocab_size)}')
print (f'loss calculée {loss}')

torch.Size([32, 101])
loss attendue 4.61512051684126
loss calculée 5.20951509475708


Pour utiliser le modèle en prédiction, il faut lui fournir un premier caractère pour amorcer la séquence : c'est le prompt. Dans notre cas, on peut initialiser la génération avec le caractère de retour à la ligne pour débuter une nouvelle phrase.

Questions :
> * Créer un prompt avec un tenseur de taille (1,1) contenant l'entier correspondant au caractère `\n`.
> * Générer une séquence de caractères de taille 100 à partir de ce prompt avec les fonctions `m.generate` et `decode`.
> * Comment est la phrase générée ?

In [48]:
print (encode(['\n']))
## YOUR CODE HERE
prompt = torch.tensor([[encode(['\n'])[0]]], dtype=torch.long).to(device)

generated_sequence = m.generate(prompt, 100)
generated_text = decode(generated_sequence[0].cpu().numpy())
print(generated_text)
print(len(generated_text))
###

[79]

,rzë 3iQE]L8
Hù«t72p2;S9:TmqiÆ»5B(;c1bÎxE
êwP
1'
whBU7gt
vQLQxDô?PT?I4Â94à!OîB4g!Q1â;»Àd
bq;YML;6sËÔ
101


La phrase générée peut varier à chaque exécution en raison de la fonction m.generate qui génère des séquences aléatoires à partir du prompt. on peut rien comprendre du sequence


### Entrainement

Pour l'entrainement, nous utilisons un optimiseur [AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) avec un learning rate de 1e-3. Une itération d'apprentissage consiste en 
- générer un batch
- appliquer le réseau de neurones (forward) et calculer la loss (`model(xb, yb)`)
- calculer le gradient (après avoir remis à zero le gradient cumulé) (`loss.backward()`)
- mettre à jour les paramètres (`optimizer.step()`).

In [49]:
max_iters = 100
batch_size = 4
eval_interval = 10
learning_rate = 1e-3
eval_iters = 20

@torch.no_grad() # no gradient is computed here
def estimate_loss():
    """ Estimate the loss on eval_iters batch of train and val sets."""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# re-create the model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 5.1185, val loss 5.0434
step 10: train loss 5.0496, val loss 4.9579
step 20: train loss 4.9822, val loss 5.0037
step 30: train loss 5.0121, val loss 5.0045
step 40: train loss 4.9834, val loss 4.9856
step 50: train loss 5.1149, val loss 5.0176
step 60: train loss 5.0777, val loss 5.0228
step 70: train loss 4.9910, val loss 5.0236
step 80: train loss 5.0666, val loss 4.9326
step 90: train loss 5.0692, val loss 5.0292


In [50]:
max_iters = 1000
batch_size = 4
eval_interval = 10
learning_rate = 1e-3
eval_iters = 20

@torch.no_grad() # no gradient is computed here
def estimate_loss():
    """ Estimate the loss on eval_iters batch of train and val sets."""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# re-create the model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.0323, val loss 4.9814
step 10: train loss 5.0138, val loss 4.9771
step 20: train loss 5.1295, val loss 5.0212
step 30: train loss 4.9969, val loss 4.9057
step 40: train loss 5.0759, val loss 4.9549
step 50: train loss 5.0276, val loss 4.9087
step 60: train loss 4.9761, val loss 4.9107
step 70: train loss 5.0188, val loss 4.8867
step 80: train loss 5.0327, val loss 4.8809
step 90: train loss 4.9382, val loss 4.8677
step 100: train loss 4.9821, val loss 4.8215
step 110: train loss 4.9293, val loss 4.7812
step 120: train loss 5.0563, val loss 4.8104
step 130: train loss 4.8460, val loss 4.8459
step 140: train loss 4.9945, val loss 4.7879
step 150: train loss 4.9849, val loss 4.8594
step 160: train loss 4.9342, val loss 4.8459
step 170: train loss 4.8429, val loss 4.8548
step 180: train loss 4.9496, val loss 4.7277
step 190: train loss 4.8580, val loss 4.7789
step 200: train loss 4.9324, val loss 4.9205
step 210: train loss 4.9335, val loss 4.7455
step 220: train loss 

In [51]:
max_iters = 10000
batch_size = 4
eval_interval = 10
learning_rate = 1e-3
eval_iters = 20

@torch.no_grad() # no gradient is computed here
def estimate_loss():
    """ Estimate the loss on eval_iters batch of train and val sets."""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# re-create the model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.1121, val loss 5.1617
step 10: train loss 5.1494, val loss 5.1465
step 20: train loss 5.1247, val loss 5.1272
step 30: train loss 5.1024, val loss 5.1324
step 40: train loss 5.1449, val loss 5.1369
step 50: train loss 5.0739, val loss 5.1670
step 60: train loss 5.0964, val loss 5.1730
step 70: train loss 5.0481, val loss 5.1445
step 80: train loss 5.1167, val loss 4.9978
step 90: train loss 5.0502, val loss 5.0353
step 100: train loss 5.0330, val loss 5.0549
step 110: train loss 5.0050, val loss 5.0526
step 120: train loss 5.0303, val loss 5.0604
step 130: train loss 4.9530, val loss 5.0674
step 140: train loss 5.0150, val loss 5.0491
step 150: train loss 4.9955, val loss 5.0872
step 160: train loss 4.9692, val loss 5.0233
step 170: train loss 4.9511, val loss 5.0038
step 180: train loss 5.0165, val loss 4.9985
step 190: train loss 5.0074, val loss 5.0482
step 200: train loss 4.9063, val loss 4.9574
step 210: train loss 4.9481, val loss 4.9995
step 220: train loss 

step 1810: train loss 3.8295, val loss 4.0000
step 1820: train loss 3.8493, val loss 3.9573
step 1830: train loss 3.7747, val loss 3.9453
step 1840: train loss 3.8230, val loss 3.8875
step 1850: train loss 3.8478, val loss 3.9433
step 1860: train loss 3.8358, val loss 3.9276
step 1870: train loss 3.7966, val loss 3.9142
step 1880: train loss 3.8630, val loss 3.9366
step 1890: train loss 3.7321, val loss 3.8755
step 1900: train loss 3.8137, val loss 3.8305
step 1910: train loss 3.8375, val loss 3.9024
step 1920: train loss 3.7986, val loss 3.8541
step 1930: train loss 3.8132, val loss 3.8256
step 1940: train loss 3.8195, val loss 3.9461
step 1950: train loss 3.8086, val loss 3.8279
step 1960: train loss 3.7576, val loss 3.9109
step 1970: train loss 3.7621, val loss 3.7938
step 1980: train loss 3.7887, val loss 3.7444
step 1990: train loss 3.7424, val loss 3.8907
step 2000: train loss 3.6932, val loss 3.8662
step 2010: train loss 3.6939, val loss 3.7780
step 2020: train loss 3.7128, val 

step 3620: train loss 3.0861, val loss 3.2785
step 3630: train loss 3.1829, val loss 3.2722
step 3640: train loss 3.1249, val loss 3.1570
step 3650: train loss 3.1280, val loss 3.1932
step 3660: train loss 3.1327, val loss 3.2339
step 3670: train loss 3.0864, val loss 3.2136
step 3680: train loss 3.1148, val loss 3.2559
step 3690: train loss 3.1503, val loss 3.2868
step 3700: train loss 3.2276, val loss 3.2192
step 3710: train loss 3.0509, val loss 3.3422
step 3720: train loss 3.0715, val loss 3.1479
step 3730: train loss 3.1399, val loss 3.2678
step 3740: train loss 3.1229, val loss 3.2509
step 3750: train loss 3.1292, val loss 3.1071
step 3760: train loss 3.0472, val loss 3.2596
step 3770: train loss 3.0534, val loss 3.1589
step 3780: train loss 3.1030, val loss 3.3041
step 3790: train loss 3.1170, val loss 3.1823
step 3800: train loss 3.0534, val loss 3.2004
step 3810: train loss 3.0759, val loss 3.2487
step 3820: train loss 3.1828, val loss 3.2228
step 3830: train loss 3.1080, val 

step 5450: train loss 2.7957, val loss 2.8930
step 5460: train loss 2.7705, val loss 2.8108
step 5470: train loss 2.8176, val loss 2.9300
step 5480: train loss 2.7379, val loss 2.8908
step 5490: train loss 2.7627, val loss 2.8346
step 5500: train loss 2.7254, val loss 2.8731
step 5510: train loss 2.7420, val loss 2.8078
step 5520: train loss 2.7285, val loss 2.9331
step 5530: train loss 2.8080, val loss 2.8356
step 5540: train loss 2.7679, val loss 2.6858
step 5550: train loss 2.7082, val loss 2.9485
step 5560: train loss 2.7756, val loss 2.8906
step 5570: train loss 2.7646, val loss 2.8598
step 5580: train loss 2.8017, val loss 2.7740
step 5590: train loss 2.7146, val loss 2.8587
step 5600: train loss 2.7849, val loss 2.8105
step 5610: train loss 2.7954, val loss 2.7806
step 5620: train loss 2.7625, val loss 2.8080
step 5630: train loss 2.8002, val loss 2.8009
step 5640: train loss 2.7978, val loss 2.7724
step 5650: train loss 2.7061, val loss 2.9113
step 5660: train loss 2.7583, val 

step 7260: train loss 2.6125, val loss 2.7063
step 7270: train loss 2.6471, val loss 2.6710
step 7280: train loss 2.6180, val loss 2.6668
step 7290: train loss 2.6051, val loss 2.8072
step 7300: train loss 2.6176, val loss 2.6768
step 7310: train loss 2.7060, val loss 2.7711
step 7320: train loss 2.6623, val loss 2.6613
step 7330: train loss 2.5887, val loss 2.6339
step 7340: train loss 2.6399, val loss 2.6475
step 7350: train loss 2.6367, val loss 2.6934
step 7360: train loss 2.5924, val loss 2.7490
step 7370: train loss 2.5509, val loss 2.6061
step 7380: train loss 2.6353, val loss 2.7944
step 7390: train loss 2.6397, val loss 2.6027
step 7400: train loss 2.5840, val loss 2.6557
step 7410: train loss 2.5372, val loss 2.6814
step 7420: train loss 2.6586, val loss 2.6646
step 7430: train loss 2.6297, val loss 2.6316
step 7440: train loss 2.5902, val loss 2.6466
step 7450: train loss 2.5557, val loss 2.8019
step 7460: train loss 2.6429, val loss 2.7549
step 7470: train loss 2.5879, val 

step 9080: train loss 2.5105, val loss 2.5723
step 9090: train loss 2.5135, val loss 2.6725
step 9100: train loss 2.5106, val loss 2.5401
step 9110: train loss 2.5256, val loss 2.5522
step 9120: train loss 2.5150, val loss 2.6431
step 9130: train loss 2.5274, val loss 2.5154
step 9140: train loss 2.4614, val loss 2.6333
step 9150: train loss 2.5283, val loss 2.6845
step 9160: train loss 2.5853, val loss 2.4963
step 9170: train loss 2.5469, val loss 2.5241
step 9180: train loss 2.4991, val loss 2.5762
step 9190: train loss 2.5085, val loss 2.5793
step 9200: train loss 2.5072, val loss 2.5729
step 9210: train loss 2.5141, val loss 2.5608
step 9220: train loss 2.5447, val loss 2.5533
step 9230: train loss 2.5409, val loss 2.6338
step 9240: train loss 2.5125, val loss 2.5913
step 9250: train loss 2.5337, val loss 2.6305
step 9260: train loss 2.5428, val loss 2.5090
step 9270: train loss 2.5676, val loss 2.5832
step 9280: train loss 2.5718, val loss 2.5499
step 9290: train loss 2.5351, val 

Une fois le réseau entrainé pendant 100 itérations, on peut générer une séquence de caractères.

Questions :
> * Quel est l'effet de l'entraînement ?
> * Augmenter le nombre d'itérations à 1000 puis à 10,000, noter la loss obtenue et la phrase générée. Qu'observez-vous ?

L'effet de l'entraînement est de réduire la perte sur les ensembles d'entraînement et de validation. On peut voir à travers les résultats de l'entraînement que la perte sur l'ensemble d'entraînement et l'ensemble de validation tendent à diminuer au fil du temps. Cependant, il y a une certaine variabilité dans la perte au fil du temps et elle peut augmenter à certains moments, ce qui peut être dû à divers facteurs tels que l'initialisation des poids du modèle ou la taille du batch.

on observe que le loss pour le test et val diminue

In [52]:
idx = torch.ones((1,1), dtype=torch.long)*3
print (decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

îoustôÈYn'ezZn lsamph(7Etsoiesor!
Surpallabesonstode
Nha-mar mpl.
QEticiolêéves tARRêure yssupeux, gi


## Single Head Attention

Nous allons maintenant implémenter le mécanisme de base de l'attention. Pour chaque couple de mots de la séquence, ce mécanisme combine Q une *query* (l'information recherchée), K une *key* (l'information obtenue) et calcule  V une *value*, un vecteur de résultat. 

![single head attention](images/single_head_attention.png)

### Masquage
Cependant, comme nous utilisons le modèle pour générer des séquences, on ne doit pas utiliser les caractères situés après le caractère courant, car ce sont justement ces caractères que l'on cherche à prédire lors de l'apprentissage : *le futur n'est pas utilisé pour prédire (le futur).* 

On va donc intégrer une matrice de masquage dans le processus. Cette matrice indique que pour le premier caractère de la séquence, on ne peux utiliser que ce caractère pour prédire (pas de contexte). Pour le second caractère, on peut utiliser le premier caractère et le second. Pour le troisième caractère, on peut utiliser le premier, le second et le troisième et ainsi de suite. Cette matrice est donc une matrice triangulaire inférieure. Cette matrice est normalisée par ligne (les lignes somment à 1).

In [53]:
T = 8

# first version of the contraints with matrix multiplication
# create a lower triangular matrix
weights0 = torch.tril(torch.ones(T,T))
# normalize each row
weights0 = weights0 / weights0.sum(1, keepdim=True) 
print (weights0)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


La couche [`softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) est une autre manière de réaliser la normalisation :

Question :
> * Vérifier qu'on obtient bien la même matrice.

In [55]:
tril = torch.tril(torch.ones(T,T))
weights = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril== 0, float('-inf'))
weights = nn.functional.softmax(weights, dim=-1)
print (weights)
print(weights==weights0)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
 

### Implémentation

Nous pouvons maintenant implémenter la couche d'attention :

![attention_formula](images/attention_formula.png)

Questions :

> * Créer les couches key, query et value comme des couches linéaires de dimension `C` x `head_size`.
> * Appliquer les couches à `x`.
> * `weights = query x key` (transposer les deuxième et troisième dimensions de key pour pouvoir faire le produit).
> * Appliquer le facteur de normalisation.
> * Appliquer le masque triangulaire et la softmax à `weights`.
> * Appliquer value à `x`.
> * Le résultat `out` est la multiplication de `weights` par `value(x)`.


In [56]:
import torch

head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Define the Key, Query, and Value layers as linear layers with dimension C x head_size
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)

# Apply each layer to the input
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)
v = value(x)  # (B, T, head_size)

# Compute the normalized product between Q and K
weights = q @ k.transpose(1, 2)  # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

# Apply the mask (lower triangular matrix)
mask = torch.tril(torch.ones(T, T))
mask = mask.expand(B, T, T)
weights = weights.masked_fill(mask == 0, float('-inf'))

# Apply the softmax
weights = torch.nn.functional.softmax(weights, dim=-1)

# Compute the output
out = weights @ v  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)

# Print the result
print(weights[0])
print(out[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8750, 0.1250, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2203, 0.4497, 0.3300, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0759, 0.7291, 0.0692, 0.1257, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1548, 0.0772, 0.0556, 0.6432, 0.0692, 0.0000, 0.0000, 0.0000],
        [0.1108, 0.6689, 0.1265, 0.0321, 0.0524, 0.0092, 0.0000, 0.0000],
        [0.1096, 0.2339, 0.0500, 0.1669, 0.2702, 0.0934, 0.0760, 0.0000],
        [0.0062, 0.1757, 0.1734, 0.0784, 0.0599, 0.0521, 0.4429, 0.0113]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.9420,  0.0448,  0.4026,  0.7188, -0.0675, -0.4895, -0.3085, -0.5384,
         -0.2700, -1.5483,  0.4122,  1.2404,  0.6455,  0.4919, -0.6181,  0.0471],
        [ 0.7881,  0.1305,  0.3204,  0.6477,  0.0201, -0.4438, -0.3201, -0.5906,
         -0.1750, -1.3614,  0.3945,  1.2050,  0.6964,  0.4644, -0.5524, -0.0143],
        [ 0.2338,  0.2372, -0.1209,  0.2518,  0

Questions :

> * Copier votre code dans `gpt_single_head.py` : la définition des couches dans le constructeur de la classe `Head` et les calculs dans la fonction `forward`.
> * Faire un entrainement.
> * Quelle loss en train et val obtenez vous ? Le texte vous parait-il meilleur ?


0.009989 M parameters   
step 0: train loss 4.7054, val loss 4.6731   
step 500: train loss 2.7430, val loss 2.8467   
step 1000: train loss 2.4996, val loss 2.5927   
step 1500: train loss 2.4571, val loss 2.5604   
step 2000: train loss 2.4131, val loss 2.5589   
step 2500: train loss 2.3903, val loss 2.5651   
step 3000: train loss 2.3752, val loss 2.5359   
step 3500: train loss 2.3536, val loss 2.5113   
step 4000: train loss 2.3521, val loss 2.4266   
step 4500: train loss 2.3412, val loss 2.4794   
step 4999: train loss 2.3423, val loss 2.4739   
'orsure!   
Qu'é, 4isoncoireyes, à cerice l,   
Vous an;    
Sinle; me ivur D''e,   
 Quomen tran duite   
L'e,   
Sand   

the results are still not readable, but there are some correct words

## Multi-head attention

La *multi-head attention* est simplement le calcul en parallèle de plusieurs *single head attention*. Chacune des single head attention est concaténée pour créer la sortie de la multi-head attention. Dans la figure issue de l'article original, le nombre de *heads* dans le *multi-head* est `h`. Afin d'opérer des combinaisons pondérées sur la sortie de chacune des single head, une couche de calcul linéaire est ajoutée.

![multi head attention](images/multi_head_attention.png)

Le code ci-dessous crée un module de multi-head attention.
Questions :
> * Dans le constructeur, créer une liste contenant `num_heads` module `Head` en utilisant la fonction [ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) de pytorch.
> * Dans la fonction `forward`, appliquer chaque single head à l'input et concaténer le résultat en utilisant la fonction [cat](https://pytorch.org/docs/stable/generated/torch.cat.html) de pytorch.

In [58]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return out

Questions :
> * Copier le fichier `gpt_single_head.py` en `gpt_multi_head.py`.
> * Ajouter le module MultiHeadAttention dans `gpt_multi_head.py`.
> * En tête de fichier, ajouter un paramètre  `n_head = 4`.
> * Dans le module BigramLanguageModel, remplacer le module Head par un module MultiHeadAttention avec les paramètres `num_heads = n_head` et `head_size = n_embd // n_head` pour garder le même nombre de paramètres.
> Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.009989 M parameters  
step 0: train loss 4.6782, val loss 4.6303  
step 500: train loss 2.6298, val loss 2.7236  
step 1000: train loss 2.4294, val loss 2.4543  
step 1500: train loss 2.3337, val loss 2.3388  
step 2000: train loss 2.2649, val loss 2.3058  
step 2500: train loss 2.2379, val loss 2.2896  
step 3000: train loss 2.2016, val loss 2.2742  
step 3500: train loss 2.1836, val loss 2.2441  
step 4000: train loss 2.1691, val loss 2.1747  
step 4500: train loss 2.1548, val loss 2.2099  
step 4999: train loss 2.1476, val loss 2.1939  
'ors,  
Leormem, 4 moncoiren!  
Slà; qui en coires an ronde à 185ivué qu'e,  
L'onmenntéme et destre,  
Sand  

## Ajout d'une couche de calcul FeedForward


Après les couches d'attention qui collectent l'information dans la séquence, une couche de calcul est ajoutée pour combiner toutes les informations de la séquence. Cette couche est un simple Multi-Layer-Perceptron avec une couche cachée et une non linéarité de type [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html).
![multi feedfoward](images/multi_ff.png)

In [ ]:
class FeedForward(nn.Module):
    """ a simple MLP with RELU """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

Questions :
> * Ajouter le module `FeedForward` dans votre fichier `gpt_multi_head.py`.
> * Ajouter cette couche `FeedForward` après la *multi-head attention*.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.010949 M parameters
step 4999: train loss 2.1290, val loss 2.1216

0.011045 M parameters  
step 0: train loss 4.6931, val loss 4.6816  
step 500: train loss 2.6344, val loss 2.7173  
step 1000: train loss 2.4309, val loss 2.4594  
step 1500: train loss 2.3400, val loss 2.3932  
step 2000: train loss 2.2804, val loss 2.3035  
step 2500: train loss 2.2491, val loss 2.2958  
step 3000: train loss 2.2197, val loss 2.2729  
step 3500: train loss 2.1930, val loss 2.2608  
step 4000: train loss 2.1813, val loss 2.2159  
step 4500: train loss 2.1611, val loss 2.1886  
step 4999: train loss 2.1533, val loss 2.1926  
'e au nossième!  
Et du! itpes,  
Et drore, le feure ghargennt tond croribrye, nes.  
Ent l'afffffoe Bantt  



## Empiler les blocs

Le réseau construit jusqu'à présent n'est en fait qu'un bloc du réseau final. Il est maintenant possible d'empiler les blocs de *multi-head attention* pour créer un réseau profond. 

![multi feedfoward](images/multi_bloc.png)


Le code suivant crée un bloc : 

In [ ]:
class Block(nn.Module):
    """ A single bloc of multi-head attention """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x

Questions :
> * Ajouter le module `Block` dans `gpt_multi_head.py`.
> * Modifier le code de `BigramLanguageModel` pour ajouter 3 `Block(n_embd, n_head=4)` avec un container [Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) à la place de `MultiHeadAttention`et `FeedForward`.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.019205 M parameters
step 4999: train loss 2.2080, val loss 2.2213

0.019493 M parameters  
step 0: train loss 4.6177, val loss 4.6225   
step 500: train loss 3.1173, val loss 3.1598   
step 1000: train loss 2.6441, val loss 2.6304   
step 1500: train loss 2.5116, val loss 2.5505   
step 2000: train loss 2.4248, val loss 2.4303   
step 2500: train loss 2.3718, val loss 2.3863   
step 3000: train loss 2.3184, val loss 2.3370   
step 3500: train loss 2.2819, val loss 2.3039   
step 4000: train loss 2.2537, val loss 2.2584   
step 4500: train loss 2.2192, val loss 2.2453   
step 4999: train loss 2.2044, val loss 2.2275   
'itreaiez sy'edboit;   
Et le biéiez le fupoure, où corresse.  
Voù, martouxs soux le l'eurit;  
D'oùt ent j  

## Amélioration de l'entraînement

Si on veut continuer à augmenter la taille du réseau, il est nécessaire d'utiliser des couches permettant d'améliorer l'entraînement et ses capacités de généralisation (réduire le sur-apprentissage). Ces couches sont :
- *skip connections* ou *residual connections*
- les couches de normalisation
- le dropout.


![multi feedfoward](images/multi_skip_norm.png)


Questions :
> * Dans le module Block, ajouter une skip connection en ajoutant l'input dans chaque connexion :
```
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
```
> * Dans le module Block, ajouter 2 couches de [LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) de taille `n_embd` avant la couche de `Multi-Head attention` et avant la `FeedForward`.
> * Après la série de 3 blocs, ajouter une couche de LayerNorm de taille `n_embd`.
> * Définir une variable `dropout = 0.2` en début de fichier et ajouter une couche de [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) :
>    * Après la couche RELU dans FeedForward
>    * Après la couche de MultiHead dans `MultiHeadAttention`
>    * Après la softmax dans la single head attention `Head`.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.019653 M parameters


0.019941 M parameters  
step 0: train loss 4.7806, val loss 4.7944  
step 500: train loss 2.4629, val loss 2.5473  
step 1000: train loss 2.2966, val loss 2.3237  
step 1500: train loss 2.2244, val loss 2.2438  
step 2000: train loss 2.1773, val loss 2.1717  
step 2500: train loss 2.1330, val loss 2.1290  
step 3000: train loss 2.1188, val loss 2.1082  
step 3500: train loss 2.0986, val loss 2.0963  
step 4000: train loss 2.0778, val loss 2.0732  
step 4500: train loss 2.0712, val loss 2.0471  
step 4999: train loss 2.0589, val loss 2.0376   
'esttRens mende, l'ouffuplavele où sué de dant da le sur cos le de;  
Quits cres, esppauis font la squa   

## Conclusion

Les principaux éléments de GPT2 sont en place, il faut maintenant faire passer le modèle à l'échelle et l'entraîner sur une base de données beaucoup plus grande. Pour comparaison, les paramètres de [GPT2](https://huggingface.co/transformers/v2.11.0/model_doc/gpt2.html) sont : 

* `vocab_size = 50257` : GPT2 modélise des tokens (subwords) alors que nous modélisons des caractères. Pour nous, `vocab_size = 100`.
* `n_positions = 1024` : la taille maximale du contexte. Pour nous, c'est `block_size = 8`.
* `n_embd = 768`:  la dimension des embeddings. Pour nous c'est `n_embd = 32`.
* `n_layer = 12`: le nombre de block. Pour nous c'est 3.
* `n_head = 12`: le nombre de multi-head attention. Pour nous c'est 4.

Au total, GPT2 est composé de 1,500 millions de paramètres et a été entrainé sur 8M de pages web, soit 40 Gb de texte.


```
10.816613 M parameters
step 0: train loss 4.7847, val loss 4.7701
step 4999: train loss 0.2683, val loss 2.1161
time: 31m47.910s   
    
Le pêcheur où l'homme en peu de Carevante
Sa conter des chosses qu'en ses yoitn!

Ils sont là-hauts parler à leurs ténèbres
A ceux qu'on rêve aux oiseaux des cheveux,
Et celus qu'on tourna jamais sous le front;
Ils se disent tu mêle aux univers.
J'ai vu Jean vu France, potte; petits contempler,
Et petié calme au milibre et versait,
M'éblouissant, emportant, écoute, ingorancessible,
On meurt s'efferayait.....--Pas cont âme parle en Apparia!
```